<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/2_arxius_JOIN_def.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Cargar el archivo principal
df_principal = pd.read_excel('/content/30.01.25 Uic.xlsx', sheet_name='Export')

# Filtrar solo ContactId 466 y 300
df_principal_subset = df_principal[df_principal['ContactId'].isin([466, 300])]

# Convertir a datetime
df_principal_subset["fecha"] = pd.to_datetime(df_principal_subset["createdAt"], format="%d/%m/%Y").dt.date

# Calcular rango de 10 segundos
start_time = df_principal_subset["createdAt"].dt.floor('10S')
end_time = start_time + pd.Timedelta(seconds=10)

df_principal_subset["rango"] = (
    start_time.dt.strftime("%H:%M:%S") + "-" + end_time.dt.strftime("%H:%M:%S")
)

# Cargar el archivo de fichas CRM
df_fichas_crm = pd.read_excel('/content/20.02.25FichasCrm (2).xlsx')

# Filtrar solo ContactId 466 y 300
df_fichas_crm_subset = df_fichas_crm[df_fichas_crm['ContactId'].isin([466, 300])]

# Convertir fechas y renombrar columna ContactId
df_fichas_crm_subset["fecha"] = pd.to_datetime(df_fichas_crm_subset["Data/hora inici"], errors="coerce", dayfirst=True)
df_fichas_crm_subset["fecha_str"] = df_fichas_crm_subset["fecha"].dt.strftime("%d/%m/%Y")
df_fichas_crm_subset = df_fichas_crm_subset.rename(columns={'Número de telèfon de l’usuari': 'ContactId2'})

# Calcular rango de 10 segundos
start_time = df_fichas_crm_subset["fecha"].dt.floor('10S')
end_time = start_time + pd.Timedelta(seconds=10)

df_fichas_crm_subset["rango2"] = (
    start_time.dt.strftime("%H:%M:%S") + "-" + end_time.dt.strftime("%H:%M:%S")
)

# Cargar el archivo de tipificaciones
df_tipificaciones = pd.read_excel('/content/20.02.25TipificacionesConversaciones.xlsx')

# Filtrar solo ContactId 466 y 300
df_tipificaciones_subset = df_tipificaciones[df_tipificaciones['ContactId'].isin([466, 300])]

# Renombrar columnas
df_tipificaciones_subset = df_tipificaciones_subset.rename(columns={'ContactId': 'ContactId1'})

# Realizar el primer LEFT JOIN entre df_principal_subset y df_tipificaciones_subset
df_merged = pd.merge(
    df_principal_subset,
    df_tipificaciones_subset,
    how='left',
    left_on=['OpenchannelInteractionId', 'ContactId'],
    right_on=['id', 'ContactId1']
)


df_merged2 = pd.merge(
    df_principal_subset,
    df_fichas_crm_subset,
    how='left',
    left_on=['ContactId'],
    right_on=[ 'ContactId']

)


df_merged.to_excel('df_merged.xlsx', index=False)
df_merged2.to_excel('df_merged2.xlsx', index=False)


import pandas as pd

# Cargar los archivos
df_merged = pd.read_excel("/content/df_merged.xlsx")
df_merged2 = pd.read_excel("/content/df_merged2.xlsx")

# --- Asegurar formatos consistentes ---
# Convertir todas las fechas a datetime
df_merged["fecha"] = pd.to_datetime(df_merged["fecha"])
# Changed line: specify the correct format for 'fecha_str'
df_merged2["fecha_str"] = pd.to_datetime(df_merged2["fecha_str"], format="%d/%m/%Y")

# Normalizar rangos de tiempo (ej: "14:30:20-14:30:30")
df_merged2["rango2"] = df_merged2["rango2"].str.replace(" ", "").str.strip()

# --- Realizar el JOIN con columnas alineadas ---
# Changed lines: Rename 'rango' in df_merged and 'rango2' in df_merged2 to avoid duplicate column names
df_final = pd.merge(
    df_merged.rename(columns={"rango": "rango_merged"}),
    df_merged2.rename(columns={
        "fecha_str": "fecha",  # Renombrar columna para igualar nombres
        "rango2": "rango_merged2"
    }),
    left_on=["fecha", "rango_merged", "ContactId"],
    right_on=["fecha", "rango_merged2", "ContactId"],  # Ahora las columnas tienen los mismos nombres
    how="left"
)

# Guardar en una ruta válida
df_final.to_excel("/content/df_final.xlsx", index=False)

# Mostrar resultado
print("Filas en df_final:", len(df_final))
print(df_final.head())
